In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
from IPython.display import clear_output
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from datasets import Dataset
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from evaluate import load
import datetime
import os
# from sklearn.model_selection import train_test_split


2024-12-07 13:15:04.739451: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 13:15:04.751043: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 13:15:04.754494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 13:15:04.763289: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-07 13:15:08.013726: W tensorflow/compiler/tf2

In [4]:
TARGETSCAN_DATASET_PATH = "/sci/nosnap/michall/roeizucker/token_classification/targetscan_RNA_combined_nrows_90000_hg38.csv"
KMER_SIZE = 6
model_name = "InstaDeepAI/nucleotide-transformer-500m-1000g"
MAX_TOKEN = 1000
BLANK_LABLE_VALUE = -100
ADD_BLANKS_TO_END = False
TRAIN_AMOUNT = 0.005
EPOCH_NUM = 10
METRIC = "None"

In [3]:
# spliceAI data:
TRAIN_DF_PATH = "/sci/nosnap/michall/roeizucker/token_classification/spliceai_train.csv"
TEST_DF_PATH = "/sci/nosnap/michall/roeizucker/token_classification/spliceai_test.csv"
train_df = pd.read_csv(TRAIN_DF_PATH)
test_df = pd.read_csv(TEST_DF_PATH)
SPLICEAI_LABLES = ['none', 'start','end']

SPLICE_AI = True
# 

In [5]:
def convert_list_representation_to_list(df,seq_col_name="sequnces",labels_col_name="labels"):
    df["result"] = df[seq_col_name].str.replace("'","")
    df["result"] = df["result"].apply(lambda x:x.strip('][').split(', '))
    df["stags"] = df[labels_col_name].apply(lambda x:x.strip('][').replace("'","").split(', '))
    df['stags'] = df['stags'].apply(lambda x: list(map(SPLICEAI_LABLES.index, x)))
    return df
if SPLICE_AI == True:
    train_df = convert_list_representation_to_list(train_df)
    test_df = convert_list_representation_to_list(test_df)

In [6]:
def create_sequnces_from_df(df):
    sequences = []
    labels = []
    for row_idx, row in df.iterrows():
        if row_idx % 100 == 0:
            clear_output(wait=True)
            # print(filename.split("/")[-1])
            print(str(int(100*(row_idx / len(df)))) + "%")

        curr_label = []
        for i, seq in enumerate(row["result"]):
            curr_label.extend([row["stags"][i]]*len(seq))
        
        label = np.zeros(len(curr_label)//KMER_SIZE,dtype=np.int64)
        for i in range(0,len(curr_label),KMER_SIZE):
            # if i+KMER_SIZE <len(curr_label) and 1 in curr_label[i:i+KMER_SIZE]) :
            #     label[i//KMER_SIZE:i//6+1] = 1
            
            for j in range(1,len(SPLICEAI_LABLES)):
                if i+KMER_SIZE <len(curr_label) and j in curr_label[i:i+KMER_SIZE]:
                    label[i//KMER_SIZE:i//KMER_SIZE+1] = j
                    break
            else: # unneeded
                label[i//KMER_SIZE:i//KMER_SIZE+1] = 0
        seq_str = "".join(row["result"])[:len(curr_label)]
        last_i = 0
        
        for i in range(0,len(label),MAX_TOKEN):
            if sum(label[i:i+MAX_TOKEN]) == 0:
                last_i = i
                continue
            sequences.append(seq_str[i*6:(i+MAX_TOKEN)*6])
            labels.append(label[i:i+MAX_TOKEN])
            last_i = i
        if ADD_BLANKS_TO_END:
            sequences.append(seq_str[last_i*6:])
            
            labels.append(np.concatenate(
                [label[last_i:] ,  
                np.array(
                    [BLANK_LABLE_VALUE]* 
                    (MAX_TOKEN - len(label[last_i:])))]))
    return sequences, labels
train_seq, train_labels = create_sequnces_from_df(train_df)

99%


In [7]:
train_seq = train_seq[:int(len(train_seq) * TRAIN_AMOUNT)]
train_labels = train_labels[:int(len(train_labels) * TRAIN_AMOUNT)]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)